In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from erutils.command_line_interface import *

In [20]:
batch = 4
chunk = 8
channel = 32
head_size = 16
B, T, C = batch, chunk, channel

wei = torch.tril(torch.ones(T, T))
x = torch.randn(B, T, C)

In [21]:
fprint(x)
fprint(wei)
fprint(x.shape)
fprint(wei.shape)

tensor([[[-1.3889,  1.7322,  0.4554,  ...,  1.1413, -1.2619, -0.5461],
         [ 0.9769, -1.5209,  0.5069,  ..., -0.7169, -0.5230, -0.4829],
         [-0.5189,  0.3361, -0.1405,  ..., -0.3845,  0.2516,  0.0395],
         ...,
         [-1.2422, -1.3158, -0.6333,  ...,  1.6936, -0.7376,  0.2938],
         [ 1.1602, -0.9791,  0.9296,  ...,  0.2606, -0.0117,  1.8600],
         [-0.4048,  0.8249,  1.2824,  ...,  1.8931, -1.0635,  0.1363]],

        [[ 0.4437, -0.3882,  1.0796,  ...,  0.6852, -1.1848, -0.7298],
         [ 1.4268, -1.0592, -0.7368,  ...,  0.0715, -2.7828, -0.3326],
         [ 1.0280, -0.2288, -0.7574,  ..., -0.4472,  0.4489, -1.9767],
         ...,
         [-0.6573,  1.1632,  1.1656,  ...,  0.2037,  1.9560,  1.2547],
         [ 0.9776,  0.6024,  0.3535,  ...,  1.3581, -0.4072,  0.5468],
         [-0.7350, -1.4649, -0.5454,  ..., -1.4068,  0.0130, -0.1184]],

        [[-0.5354, -1.7225, -0.1682,  ...,  1.1589, -1.4658,  1.0281],
         [ 0.2916, -1.7385, -0.2498,  ..., -0

In [10]:
wei_u = wei / wei.sum(1, keepdim=True)

In [11]:
wei_u

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [12]:
out = wei_u @ x

In [15]:
out[0]

tensor([[-0.1375,  1.4464],
        [ 0.3996,  1.1857],
        [ 0.3245,  0.5956],
        [ 0.3577,  0.4832],
        [ 0.6910,  0.3519],
        [ 0.9176,  0.2630],
        [ 0.6695,  0.0588],
        [ 0.3012,  0.0066]])

In [38]:
x = torch.randn(B, T, C)

In [33]:
x.shape

torch.Size([4, 8, 32])

In [34]:
q = query(x)
k = key(x)
wei = q @ k.transpose(-2, -1)  # (B,T,C) @ (B,C,T) -> (B,T,T)

In [35]:
wei[0]

tensor([[ 1.5245,  0.2666, -0.9527,  0.3459,  0.0797, -1.1958, -0.3120,  0.3097],
        [ 0.7752, -1.1551, -0.6436, -0.4966,  0.4932, -1.2287,  0.3951,  0.7215],
        [ 0.5075,  1.0929,  1.3904,  0.0424,  0.8485,  0.7476,  1.5912,  0.3154],
        [-0.1204,  0.5672,  0.1072, -2.5145,  0.2561, -0.8919, -1.1769,  0.4475],
        [ 1.4162, -0.1153,  0.6543, -1.4144, -0.1589, -0.6951, -0.8311, -0.6641],
        [ 0.0591,  0.6327, -0.6453, -0.5096,  1.7258, -1.8088,  0.1996,  0.9214],
        [ 0.1479, -0.4396,  0.1087, -0.2881,  0.3137, -0.0395,  0.6278, -0.1509],
        [-0.0097,  0.3331, -1.5143,  1.2115,  0.1943,  0.2910, -1.4325,  0.5627]],
       grad_fn=<SelectBackward0>)

In [36]:
trill = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8733, 0.1267, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1918, 0.3444, 0.4638, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2306, 0.4587, 0.2896, 0.0210, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5131, 0.1109, 0.2395, 0.0303, 0.1062, 0.0000, 0.0000, 0.0000],
        [0.1077, 0.1911, 0.0533, 0.0610, 0.5703, 0.0166, 0.0000, 0.0000],
        [0.1473, 0.0819, 0.1416, 0.0952, 0.1739, 0.1221, 0.2380, 0.0000],
        [0.0942, 0.1328, 0.0209, 0.3195, 0.1156, 0.1273, 0.0227, 0.1670]],
       grad_fn=<SelectBackward0>)

In [39]:
v = value(x)

out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [40]:
out[0]

tensor([[-0.6000, -0.1009,  0.3509,  0.1668, -0.3324,  0.3932, -0.5535, -0.2136,
          0.1832, -0.3507,  0.6178,  0.1775, -0.2511, -0.1789, -0.3238,  1.0005],
        [-0.3103, -0.1107,  0.3253,  0.1473, -0.2659,  0.2882, -0.4794, -0.2161,
          0.0731, -0.2645,  0.5403,  0.1005, -0.2625, -0.2253, -0.2109,  0.8247],
        [ 0.1917, -0.0940,  0.4344, -0.5346, -0.4362,  0.1923,  0.1826, -0.4705,
          0.0849, -0.6617,  0.0718,  0.4117,  0.1817, -0.2740,  0.9764,  0.1610],
        [ 0.4670, -0.1253,  0.3523, -0.3214, -0.2556,  0.0531,  0.0624, -0.3717,
         -0.0972, -0.3683,  0.1257,  0.1571, -0.0037, -0.3361,  0.6970,  0.1196],
        [-0.2675, -0.0369,  0.3093, -0.2212, -0.2785,  0.2922, -0.1058, -0.2840,
          0.2578, -0.5422,  0.2064,  0.2292, -0.0650, -0.0217,  0.3267,  0.5272],
        [ 0.1776,  0.2427, -0.1996, -0.0755,  0.4251,  0.0276,  0.1332,  0.0411,
          0.4370, -0.2521, -0.4658, -0.3974, -0.4314,  0.7762,  0.2214,  0.0574],
        [-0.1019,  0.1

In [41]:
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

In [42]:
x = torch.randn((B, T, C))
q = query(x)
k = key(x)
v = value(x)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5
trill = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
out = wei @ v


In [44]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5829, 0.4171, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4209, 0.3279, 0.2512, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4538, 0.1940, 0.1328, 0.2194, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1986, 0.1427, 0.1454, 0.3062, 0.2070, 0.0000, 0.0000, 0.0000],
        [0.1878, 0.1488, 0.1570, 0.2069, 0.1894, 0.1101, 0.0000, 0.0000],
        [0.0803, 0.1554, 0.1862, 0.1992, 0.1740, 0.1150, 0.0899, 0.0000],
        [0.1765, 0.1092, 0.0981, 0.1217, 0.1159, 0.0739, 0.2036, 0.1011]],
       grad_fn=<SelectBackward0>)

In [2]:
number_of_embedded = 32
number_of_heads = 16
dp1 = nn.Dropout(0.2)
dp2 = nn.Dropout(0.2)
attn = nn.Linear(number_of_embedded, 3 * number_of_embedded)
proj = nn.Linear(number_of_embedded, number_of_embedded)

In [3]:
chunk = 64
x = torch.randn(10, chunk, number_of_embedded)
B, T, C = x.shape

In [4]:
x = attn(x)
print('xShape ::: ', x.shape)
q, k, v = x.split(number_of_embedded, dim=2)

xShape :::  torch.Size([10, 64, 96])


In [5]:
q.shape

torch.Size([10, 64, 32])

In [6]:
q.shape


torch.Size([10, 64, 32])

In [7]:
q = q.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)
k = k.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)
v = v.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)

In [8]:
v.shape
import math


In [15]:
att = q @ k.transpose(-2, -1) * (1.0 / math.sqrt(k.size(0)))
att.shape

torch.Size([10, 16, 64, 64])

In [16]:
bias = torch.tril(torch.ones(T, T).view(1, 1, chunk, chunk))
att = att.masked_fill(bias[:, :, :T, :T] == 0, float('-inf'))
att = F.softmax(att, dim=-1)

In [17]:
att = dp1(att)

In [18]:
wei = att @ v

In [19]:
wei = wei.transpose(1,2).contiguous().view(B,T,C)

In [20]:
wei.shape

torch.Size([10, 64, 32])